## Bundesliga German Football Table Web Scraping

In [2]:
# Import libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://www.bundesliga.com/en/bundesliga/table"

response = requests.get(url)

response.status_code

200

## Get Soup

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

From the soup object, the table in the Bundesliga page can be found in the table tag. The rows can be found with the `tr` tags when you use Inspect from right clicking a page.

In [5]:
results = soup.find('table', {'class': 'table'}).find_all('tr')

In [6]:
# Remove first row (header):

results = results[1:]

In [7]:
# Number of teams should be 18
len(results)

18

## Get Team Name

The team name can found from the `td` tag with the class name as `team` then the `a` tag and then the `span` tag with the class of `d-none d-lg-inline`.

In [8]:
results[0].find('td', {'class': 'team'}).find('span', {'class': 'd-none d-lg-inline'}).get_text()

'FC Bayern München'

In [9]:
results[17].find('td', {'class': 'team'}).find('span', {'class': 'd-none d-lg-inline'}).get_text()

'SpVgg Greuther Fürth'

## Number Of Matches

The class of interest is `matches` which is from the `td` tags.

In [10]:
results[0].find('td', {'class': 'matches'}).get_text()

'22'

## Points

In [11]:
results[0].find('td', {'class': 'pts'}).get_text()

'52'

## Wins, Draws, Losses

In [12]:
# Wins:

results[0].find('td', {'class': 'd-none d-lg-table-cell wins'}).get_text()

'17'

In [13]:
# Draws:

results[0].find('td', {'class': 'd-none d-lg-table-cell draws'}).get_text()

'1'

In [14]:
# Losses

results[0].find('td', {'class': 'd-none d-lg-table-cell looses'}).get_text()

'4'

## Goals

In [15]:
results[0].find('td', {'class': 'd-none d-md-table-cell goals'}).get_text()

'70:25'

## Goal Difference

In [16]:
# Remove plus sign cases

results[0].find('td', {'class': 'difference'}).get_text().replace("+", "")

'45'

In [17]:
results[17].find('td', {'class': 'difference'}).get_text().replace("+", "")

'-37'

## Creating Table

In [18]:
# Webscrape the table, I use list comprhension instead of for loop append method:

teams = [result.find('td', {'class': 'team'}).find('span', {'class': 'd-none d-lg-inline'}).get_text() 
         for result in results]
    
matches = [result.find('td', {'class': 'matches'}).get_text() for result in results]

points = [result.find('td', {'class': 'pts'}).get_text() for result in results]

wins = [result.find('td', {'class': 'd-none d-lg-table-cell wins'}).get_text() for result in results]

draws = [result.find('td', {'class': 'd-none d-lg-table-cell draws'}).get_text() for result in results]

losses = [result.find('td', {'class': 'd-none d-lg-table-cell looses'}).get_text() for result in results]

goals = [result.find('td', {'class': 'd-none d-md-table-cell goals'}).get_text() for result in results]

goal_diff = [result.find('td', {'class': 'difference'}).get_text().replace("+", "") for result in results]

In [19]:
## Make pandas Dataframe:

bundesliga_df = pd.DataFrame({'Rank': range(1, 19), 'Team': teams, 'Matches': matches,
                              'Points': points, 'Wins': wins, 'Draws': draws,
                              'Losses': losses, 'Goals': goals, 'Goal Difference': goal_diff})

In [20]:
bundesliga_df

,Rank,Team,Matches,Points,Wins,Draws,Losses,Goals,Goal Difference
0,1,FC Bayern München,22,52,17,1,4,70:25,45
1,2,Borussia Dortmund,22,46,15,1,6,57:36,21
2,3,Bayer 04 Leverkusen,22,41,12,5,5,58:36,22
3,4,RB Leipzig,22,34,10,4,8,43:27,16
4,5,TSG Hoffenheim,22,34,10,4,8,43:34,9
5,6,SC Freiburg,22,34,9,7,6,34:25,9
6,7,1. FC Union Berlin,22,34,9,7,6,29:30,-1
7,8,1. FC Köln,22,32,8,8,6,34:37,-3
8,9,1. FSV Mainz 05,22,31,9,4,9,31:24,7
9,10,Eintracht Frankfurt,22,31,8,7,7,33:34,-1


In [21]:
## Split Goals Into Goals For & Goals Against:

bundesliga_df[['Goals For','Goals Against']] = bundesliga_df['Goals'].str.split(":",expand=True,)

In [22]:
bundesliga_df

,Rank,Team,Matches,Points,Wins,Draws,Losses,Goals,Goal Difference,Goals For,Goals Against
0,1,FC Bayern München,22,52,17,1,4,70:25,45,70,25
1,2,Borussia Dortmund,22,46,15,1,6,57:36,21,57,36
2,3,Bayer 04 Leverkusen,22,41,12,5,5,58:36,22,58,36
3,4,RB Leipzig,22,34,10,4,8,43:27,16,43,27
4,5,TSG Hoffenheim,22,34,10,4,8,43:34,9,43,34
5,6,SC Freiburg,22,34,9,7,6,34:25,9,34,25
6,7,1. FC Union Berlin,22,34,9,7,6,29:30,-1,29,30
7,8,1. FC Köln,22,32,8,8,6,34:37,-3,34,37
8,9,1. FSV Mainz 05,22,31,9,4,9,31:24,7,31,24
9,10,Eintracht Frankfurt,22,31,8,7,7,33:34,-1,33,34


In [23]:
# Drop Goals column and rearrange columns:

bundesliga_df.drop('Goals', axis = 1, inplace = True)

bundesliga_df

,Rank,Team,Matches,Points,Wins,Draws,Losses,Goal Difference,Goals For,Goals Against
0,1,FC Bayern München,22,52,17,1,4,45,70,25
1,2,Borussia Dortmund,22,46,15,1,6,21,57,36
2,3,Bayer 04 Leverkusen,22,41,12,5,5,22,58,36
3,4,RB Leipzig,22,34,10,4,8,16,43,27
4,5,TSG Hoffenheim,22,34,10,4,8,9,43,34
5,6,SC Freiburg,22,34,9,7,6,9,34,25
6,7,1. FC Union Berlin,22,34,9,7,6,-1,29,30
7,8,1. FC Köln,22,32,8,8,6,-3,34,37
8,9,1. FSV Mainz 05,22,31,9,4,9,7,31,24
9,10,Eintracht Frankfurt,22,31,8,7,7,-1,33,34


In [24]:
bundesliga_df = bundesliga_df.reindex(columns=['Rank', 'Team', 'Matches', 'Points',
                                               'Wins', 'Draws', 'Losses', 'Goals For',
                                               'Goals Against', 'Goal Difference'])

In [25]:
bundesliga_df

,Rank,Team,Matches,Points,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference
0,1,FC Bayern München,22,52,17,1,4,70,25,45
1,2,Borussia Dortmund,22,46,15,1,6,57,36,21
2,3,Bayer 04 Leverkusen,22,41,12,5,5,58,36,22
3,4,RB Leipzig,22,34,10,4,8,43,27,16
4,5,TSG Hoffenheim,22,34,10,4,8,43,34,9
5,6,SC Freiburg,22,34,9,7,6,34,25,9
6,7,1. FC Union Berlin,22,34,9,7,6,29,30,-1
7,8,1. FC Köln,22,32,8,8,6,34,37,-3
8,9,1. FSV Mainz 05,22,31,9,4,9,31,24,7
9,10,Eintracht Frankfurt,22,31,8,7,7,33,34,-1


## All In One Function

Putting it all together in one function. I did not include saving the table into a .csv or Excel file. All the values are strings. The user would have to convert them into numeric if they want to.

In [26]:
def bundesliga_scrape():
    
    url = "https://www.bundesliga.com/en/bundesliga/table"

    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results = soup.find('table', {'class': 'table'}).find_all('tr')
    
    # Remove first row:

    results = results[1:]
    
    # Webscrape the table, I use list comprhension instead of for loop append method:

    teams = [result.find('td', {'class': 'team'}).find('span', {'class': 'd-none d-lg-inline'}).get_text() 
             for result in results]  
    matches = [result.find('td', {'class': 'matches'}).get_text() for result in results]
    points = [result.find('td', {'class': 'pts'}).get_text() for result in results]

    wins = [result.find('td', {'class': 'd-none d-lg-table-cell wins'}).get_text() for result in results]
    draws = [result.find('td', {'class': 'd-none d-lg-table-cell draws'}).get_text() for result in results]
    losses = [result.find('td', {'class': 'd-none d-lg-table-cell looses'}).get_text() for result in results]

    goals = [result.find('td', {'class': 'd-none d-md-table-cell goals'}).get_text() for result in results]
    goal_diff = [result.find('td', {'class': 'difference'}).get_text().replace("+", "") for result in results]
    
    ## Make pandas Dataframe:

    df = pd.DataFrame({'Rank': range(1, 19), 'Team': teams, 'Matches': matches,
                              'Points': points, 'Wins': wins, 'Draws': draws,
                              'Losses': losses, 'Goals': goals, 'Goal Difference': goal_diff})
    
    # Split Goals Into Goals For & Goals Against:
    df[['Goals For','Goals Against']] = df['Goals'].str.split(":",expand=True,)
    
    # Drop Goals column
    df.drop('Goals', axis = 1, inplace = True)
    
    # Rearrange columns
    df = df.reindex(columns=['Rank', 'Team', 'Matches', 'Points',
                             'Wins', 'Draws', 'Losses', 'Goals For',
                             'Goals Against', 'Goal Difference'])
    return df

In [27]:
# Function Call

bundesliga_df = bundesliga_scrape()

bundesliga_df

,Rank,Team,Matches,Points,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference
0,1,FC Bayern München,22,52,17,1,4,70,25,45
1,2,Borussia Dortmund,22,46,15,1,6,57,36,21
2,3,Bayer 04 Leverkusen,22,41,12,5,5,58,36,22
3,4,RB Leipzig,22,34,10,4,8,43,27,16
4,5,TSG Hoffenheim,22,34,10,4,8,43,34,9
5,6,SC Freiburg,22,34,9,7,6,34,25,9
6,7,1. FC Union Berlin,22,34,9,7,6,29,30,-1
7,8,1. FC Köln,22,32,8,8,6,34,37,-3
8,9,1. FSV Mainz 05,22,31,9,4,9,31,24,7
9,10,Eintracht Frankfurt,22,31,8,7,7,33,34,-1


In [28]:
# Today's date:

from datetime import date

date.today()

datetime.date(2022, 2, 15)

In [29]:
'bundesliga_table_' + str(date.today())

'bundesliga_table_2022-02-15'

In [30]:
# Save to .csv:

bundesliga_df.to_csv('bundesliga_table_' + str(date.today()) + '.csv', index = False)